In [1]:
'''
his notebook shows how to use a pre-existing scikit-learn trained XGBoost model with the Amazon SageMaker XGBoost Algorithm
container to quickly create a hosted endpoint for that model. Please note that scikit-learn XGBoost model is compatible with 
SageMaker XGBoost container, whereas other gradient boosted tree models (such as one trained in SparkML) are not.
'''

'\nhis notebook shows how to use a pre-existing scikit-learn trained XGBoost model with the Amazon SageMaker XGBoost Algorithm\ncontainer to quickly create a hosted endpoint for that model. Please note that scikit-learn XGBoost model is compatible with \nSageMaker XGBoost container, whereas other gradient boosted tree models (such as one trained in SparkML) are not.\n'

In [2]:
%%time

import os
import boto3
import re
import json
from sagemaker import get_execution_role

region = boto3.Session().region_name
role = get_execution_role()

CPU times: user 1.1 s, sys: 124 ms, total: 1.23 s
Wall time: 9.48 s


In [3]:
#create a s3 bucket, then run:

bucket='stephenh-xgboost' # put your s3 bucket name here, and create s3 bucket

In [4]:
prefix = 'sagemaker/DEMO-xgboost-byo'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)
# customize to your bucket where you have stored the data

In [5]:
#train a scikit learn XGBoost model

In [6]:
!conda install -y -c conda-forge xgboost

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    py-xgboost-0.90            |   py36he1b5a44_2          73 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    xgboost-0.90               |   py36he1b5a44_2          11 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    libxgboost-0.90            |       he1b5a44_2         2.4 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.7 MB



In [8]:
%%time
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

CPU times: user 890 ms, sys: 319 ms, total: 1.21 s
Wall time: 2.7 s


In [9]:
#Prepare the dataset for training

In [10]:
%%time

import struct
import io
import boto3

def get_dataset():
  import pickle
  import gzip
  with gzip.open('mnist.pkl.gz', 'rb') as f:
      u = pickle._Unpickler(f)
      u.encoding = 'latin1'
      return u.load()

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 15.7 µs


In [11]:
train_set, valid_set, test_set = get_dataset()

train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]

In [13]:
print(train_X.shape)

(50000, 784)


In [14]:
train_y.shape

(50000,)

In [15]:
valid_X.shape

(10000, 784)

In [16]:
test_X.shape

(10000, 784)

In [17]:
train_X[1,1:5]

array([0., 0., 0., 0.], dtype=float32)

In [20]:
train_X[0:1,:]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [21]:
#Train the XGBClassifier

In [22]:
import xgboost as xgb
import sklearn as sk 

bt = xgb.XGBClassifier(max_depth=5,
                       learning_rate=0.2,
                       n_estimators=10,
                       objective='multi:softmax')   # Setup xgboost model
bt.fit(train_X, train_y, # Train it to our data
       eval_set=[(valid_X, valid_y)], 
       verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.2,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=10, n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [23]:
#Save the trained model file

In [24]:
model_file_name = "local-xgboost-model"
bt._Booster.save_model(model_file_name)

In [25]:
!tar czvf model.tar.gz $model_file_name

local-xgboost-model


In [26]:
#Upload the pre-trained model to S3

In [27]:
fObj = open("model.tar.gz", 'rb')
key= os.path.join(prefix, model_file_name, 'model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

In [28]:
#Set up hosting for the model

In [29]:
'''
1. Import model into hosting
'''

'\n1. Import model into hosting\n'

In [30]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

	get_image_uri(region, 'xgboost', '0.90-1').


In [31]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)
sm_client = boto3.client('sagemaker')

print (model_url)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

https://s3-us-east-1.amazonaws.com/stephenh-xgboost/sagemaker/DEMO-xgboost-byo/local-xgboost-model/model.tar.gz
arn:aws:sagemaker:us-east-1:001633343326:model/local-xgboost-model2019-10-11-17-10-53
CPU times: user 53.2 ms, sys: 4.05 ms, total: 57.2 ms
Wall time: 320 ms


In [32]:
#Create endpoint configuration

In [33]:
'''
SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. 
In order to support this, you can create an endpoint configuration, that describes the distribution of traffic 
across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration 
describes the instance type required for model deployment.
'''

'\nSageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. \nIn order to support this, you can create an endpoint configuration, that describes the distribution of traffic \nacross the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration \ndescribes the instance type required for model deployment.\n'

In [34]:
from time import gmtime, strftime

endpoint_config_name = 'DEMO-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

DEMO-XGBoostEndpointConfig-2019-10-11-17-13-21
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:001633343326:endpoint-config/demo-xgboostendpointconfig-2019-10-11-17-13-21


In [35]:
#Create endpoint

In [36]:
'''
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. 
The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes 
to complete.
'''

'\nLastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. \nThe end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes \nto complete.\n'

In [37]:
%%time
import time

endpoint_name = 'DEMO-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

DEMO-XGBoostEndpoint-2019-10-11-17-15-52
arn:aws:sagemaker:us-east-1:001633343326:endpoint/demo-xgboostendpoint-2019-10-11-17-15-52
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:001633343326:endpoint/demo-xgboostendpoint-2019-10-11-17-15-52
Status: InService
CPU times: user 165 ms, sys: 8.51 ms, total: 173 ms
Wall time: 9min 1s


In [38]:
#Create endpoint
'''
 obtain the endpoint from the client library using the result from previous operations
 and generate classifications from the model using that endpoint
'''

'\n obtain the endpoint from the client library using the result from previous operations\n and generate classifications from the model using that endpoint\n'

In [39]:
runtime_client = boto3.client('runtime.sagemaker')

In [40]:
# generate the prediction for a single datapoint

In [41]:
import numpy as np
point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")

In [42]:
%%time
import json

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
print('Predicted Class Probabilities: {}.'.format(result))

Predicted Class Probabilities: [0.023023370653390884, 0.016221631318330765, 0.020377131178975105, 0.025556255131959915, 0.017341898754239082, 0.024211231619119644, 0.017396802082657814, 0.8126810789108276, 0.017022203654050827, 0.026168374344706535].
CPU times: user 12 ms, sys: 3.97 ms, total: 16 ms
Wall time: 161 ms


In [43]:
#Post process the output

In [44]:
floatArr = np.array(json.loads(result))
predictedLabel = np.argmax(floatArr)
print('Predicted Class Label: {}.'.format(predictedLabel))
print('Actual Class Label: {}.'.format(point_y))

Predicted Class Label: 7.
Actual Class Label: 7.


In [45]:
# Delete the Endpoint
'''
remove the hosted endpoint you created and avoid any charges from a stray instance being left on.
'''

In [50]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'fe2af2e9-9312-4741-a783-c1e63b10331d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fe2af2e9-9312-4741-a783-c1e63b10331d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 11 Oct 2019 18:15:19 GMT'},
  'RetryAttempts': 0}}